# Data Exploration for the German Traffic Signs

In [1]:
import pickle
import pandas as pd

In [2]:
with open("/data/train.p", "rb") as fd:
    train = pickle.load(fd)

In [6]:
print([(key, value.shape) for key, value in train.items()])

[('coords', (34799, 4)), ('labels', (34799,)), ('features', (34799, 32, 32, 3)), ('sizes', (34799, 2))]
